# Create a Grain Boundary with Strain Matching

Create a grain boundary between two crystallographic orientations of the same material using strain matching algorithm to minimize interfacial strain.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
2. Set notebook parameters in cell 1.1 below (or use the default values).
3. Set slab parameters for both phases in cell 2.1 (or use default).
4. Set grain boundary parameters in cell 3.1 (or use default).
5. Click "Run" > "Run All" to run all cells.
6. Wait for the run to complete (depending on the parameters can take a few minutes).
7. Scroll down to view results.

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input material
2. Create slabs for both phases and select the terminations
3. Generate grain boundaries with strain matcher and plot strain vs number of atoms
4. Select the grain boundary with the desired strain and visualize it

## Notes
1. We perform strain matching on the slabs to extract the supercell dimensions.
2. When the strain matching is finished, the grain boundary with the lowest strain (and reasonable number of atoms) is selected.
3. For more information about grain boundaries, see the Introduction notebook.


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [ ]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 

# Parameters for Phase 1
PHASE_1_MILLER_INDICES = (1, 1, 1)
PHASE_1_THICKNESS = 6  # in atomic layers
PHASE_1_USE_ORTHOGONAL_Z = True

# Parameters for Phase 2
PHASE_2_MILLER_INDICES = (0, 0, 1)
PHASE_2_THICKNESS = 4  # in atomic layers
PHASE_2_USE_ORTHOGONAL_Z = True

INTERPHASE_GAP = 2.0  # in Angstrom

# Maximum area for the superlattice search algorithm
MAX_AREA = 100  # in Angstrom^2

# Parameters for the final material
SLAB_MILLER_INDICES = (0,0,1)
SLAB_THICKNESS = 4 # in atomic layers
SLAB_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
SLAB_VACUUM = 20.0  # in Angstrom

# Set the termination pair indices
TERMINATION_PAIR_INDEX = 0  # Will be overridden if interactive selection is used

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
  
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_grain_boundary.ipynb")


### 1.3. Get input material

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0]

### 1.4. Preview Material

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize([material], repetitions=[3, 3, 1], rotation="0x")

## 2. Configure slabs and select termination pair

### 2.1. Create Phase 1 and Phase 2 Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.

In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab

phase_1_configuration = SlabConfiguration(
    bulk=material,
    miller_indices=PHASE_1_MILLER_INDICES,
    thickness=PHASE_1_THICKNESS,
    use_orthogonal_z=PHASE_1_USE_ORTHOGONAL_Z
)

phase_2_configuration = SlabConfiguration(
    bulk=material,
    miller_indices=PHASE_2_MILLER_INDICES,
    thickness=PHASE_2_THICKNESS,
    use_orthogonal_z=PHASE_2_USE_ORTHOGONAL_Z
)

### 2.2. Get possible terminations for the slabs

In [ ]:
phase_1_terminations = get_terminations(phase_1_configuration)
phase_2_terminations = get_terminations(phase_2_configuration)

### 2.3. Visualize slabs for all possible terminations

In [ ]:
phase_1_slabs = [create_slab(phase_1_configuration, termination) for termination in phase_1_terminations]
phase_2_slabs = [create_slab(phase_2_configuration, termination) for termination in phase_2_terminations]

visualize([{"material":slab, "title": slab.metadata["build"]["termination"]} for slab in phase_1_slabs], repetitions=[3, 3, 1], rotation="-90x")
visualize([{"material":slab, "title": slab.metadata["build"]["termination"]} for slab in phase_2_slabs], repetitions=[3, 3, 1], rotation="-90x")

### 2.4. Print terminations for the grain boundary

In [ ]:
from itertools import product

termination_pairs = list(product(phase_1_terminations, phase_2_terminations))    
print("Termination Pairs (Phase 1, Phase 2)")
for idx, termination_pair in enumerate(termination_pairs):
    print(f"    {idx}: {termination_pair}")

### 2.5. Select termination pair for the grain boundary

In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

termination_pair_index = TERMINATION_PAIR_INDEX

termination_pair = termination_pairs[termination_pair_index]
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination_pair = await ui_prompt_select_array_element_by_index_pyodide(termination_pairs, element_name="phase 1/phase 2 termination pair")
    else:
        termination_pair = ui_prompt_select_array_element_by_index(termination_pairs, element_name="phase 1/phase 2 termination pair")

## 3. Create grain boundary

### 3.1. Initialize the Grain Boundary Configuration

In [ ]:
from mat3ra.made.tools.build.grain_boundary import SlabGrainBoundaryConfiguration

phase_1_termination, phase_2_termination = termination_pair

# Create a slab configuration for the final grain boundary structure
slab_configuration = SlabConfiguration(
    bulk=material,
    miller_indices=SLAB_MILLER_INDICES,
    thickness=SLAB_THICKNESS,
    vacuum=SLAB_VACUUM,
)

grain_boundary_configuration = SlabGrainBoundaryConfiguration(
    phase_1_configuration=phase_1_configuration,
    phase_2_configuration=phase_2_configuration,
    phase_1_termination=phase_1_termination,
    phase_2_termination=phase_2_termination,
    gap=INTERPHASE_GAP,
    slab_configuration=slab_configuration
)

### 3.2. Set Strain Matching Algorithm Parameters

In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingParameters
from mat3ra.made.tools.build.grain_boundary.builders import SlabGrainBoundaryBuilderParameters

zsl_strain_matching_parameters = ZSLStrainMatchingParameters(
    max_area=MAX_AREA
)

builder_parameters = SlabGrainBoundaryBuilderParameters(
    strain_matching_parameters=zsl_strain_matching_parameters
)


### 3.3. Generate grain boundary with strain matcher

In [ ]:
from mat3ra.made.tools.build.grain_boundary import SlabGrainBoundaryBuilder

grain_boundary_builder = SlabGrainBoundaryBuilder(build_parameters=builder_parameters)
grain_boundary = grain_boundary_builder.get_material(configuration=grain_boundary_configuration)

## 4. Preview the grain boundary

In [ ]:
visualize([grain_boundary], repetitions=[1, 1, 1])
visualize([grain_boundary], repetitions=[1, 1, 1], rotation="-90x")

## 5. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
set_materials(grain_boundary)